In [64]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import datetime     
import numpy as np
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar',
          'https://www.googleapis.com/auth/spreadsheets.readonly'
]

calendarID="c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com"

"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId=calendarID, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])



Getting the upcoming 10 events
No upcoming events found.


In [75]:
from googleapiclient.discovery import build
def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
data = pull_sheet_data(SCOPES, '1qUeAfO6dD-dXJmSllIMBAhpGK9oMCHrTCBZwDWpf0YI', 'Computer_Readable!A1:G')

COMPLETE: Data copied


In [76]:
len(data), df.shape

(199, (198, 9))

In [77]:
import pandas as pd
df = pd.DataFrame(data[1:], columns=data[0])

In [78]:
df['start']=df.Time.str.split(' - ').map(lambda x: x[0])
df['end']=df.Time.str.split(' - ').map(lambda x: x[1])

In [79]:
df

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,calendarEventID,start,end
0,8/1/2020,10:00 am - 12:00 pm,ec,ec,Operations Retreat,1,ohr0tgrvrl8dvubc9qimkgj0jk,10:00 am,12:00 pm
1,8/15/2020,10:00 am - 12:00 pm,ec,ec,Pre-kickoff EC meeting,1,ss0otj25v8c8erlgfj3ivn8dg8,10:00 am,12:00 pm
2,8/29/2020,10:30 am - 11 am,"students2023, students2022, students2021","students2023, students2022, students2021, ec",Student Welcome,8,kses5ekteino8k1qju1ra2tg04,10:30 am,11 am
3,8/29/2020,10:30 am - 12 pm,volunteers,"volunteers, ec",Volunteer Orientation,2,3h9k1q4un38gsg4h84pqq3f60k,10:30 am,12 pm
4,8/29/2020,11 am - 12 pm,students2023,students2023,Sophomore Orientation,7,p87olr1eag84ggf89m752pqhj8,11 am,12 pm
...,...,...,...,...,...,...,...,...,...
193,5/8/2021,10 am - 12 pm,Senior Enrichment,"senior-enrichment-instructors, students2021",Senior Enrichment,6,2uoh27vdlhilq3g2u2ebp1gmio,10 am,12 pm
194,5/8/2021,12 pm - 2 pm,Sophomore Mentoring,"2023mentors, students2023",Sophomore Mentoring,7,42o48dn77p1uu832nh6v5q3qo8,12 pm,2 pm
195,5/8/2021,12 pm - 2 pm,Junior Mentoring,"2022mentors, students2022",Junior Mentoring,8,sqfraf7aq8o6lgluk40fv6r2rk,12 pm,2 pm
196,5/8/2021,12 pm - 2 pm,Senior Mentoring,"2021mentors, students2021",Senior Mentoring,9,qeeerh92o11f38o57ijp83gl90,12 pm,2 pm


In [70]:
eventservice=service.events()

In [71]:
for k,row in df.iterrows():
    try:
        starttime=datetime.datetime.strptime(row.Date + " " + row.start, '%m/%d/%Y %I:%M %p')
    except ValueError:
        starttime=datetime.datetime.strptime(row.Date + " " + row.start, '%m/%d/%Y %I %p')
        
    try:
        endtime=datetime.datetime.strptime(row.Date + " " + row.end, '%m/%d/%Y %I:%M %p')
    except ValueError:
        endtime=datetime.datetime.strptime(row.Date + " " + row.end, '%m/%d/%Y %I %p')
    print(starttime, endtime)
    body={
        "description": row['Event Title'],
        "summary":  row['Event Title'],
        "start":{
            "dateTime": starttime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "end":{
            "dateTime": endtime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "colorId": f"{row.Color}",
        "guestsCanInviteOthers": False

    }
    if row.get("calendarEventID", "-")=="-":
    #    print('inserting')
        r=eventservice.insert(calendarId=calendarID,
                             body=body)
        response=r.execute()
        #print(response)
        df.loc[k, 'calendarEventID']=response['id']

2020-08-01 10:00:00 2020-08-01 12:00:00
{'kind': 'calendar#event', 'etag': '"3195790450262000"', 'id': 'ohr0tgrvrl8dvubc9qimkgj0jk', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=b2hyMHRncnZybDhkdnViYzlxaW1rZ2owamsgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:47:05.000Z', 'updated': '2020-08-20T03:47:05.131Z', 'summary': 'Operations Retreat', 'description': 'Operations Retreat', 'colorId': '1', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2020-08-01T10:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2020-08-01T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': 'ohr0tgrvrl8dvubc9qimkgj0jk@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2020-08-15 10:00:00 2020-08-15 12:00:00
{

{'kind': 'calendar#event', 'etag': '"3195790456776000"', 'id': '6krpl0pi3g7msn68rvcfgorf6c', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=NmtycGwwcGkzZzdtc242OHJ2Y2Znb3JmNmMgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:47:08.000Z', 'updated': '2020-08-20T03:47:08.388Z', 'summary': 'Senior Enrichment', 'description': 'Senior Enrichment', 'colorId': '6', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2020-09-12T10:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2020-09-12T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': '6krpl0pi3g7msn68rvcfgorf6c@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2020-09-12 12:00:00 2020-09-12 14:00:00
{'kind': 'calendar#event', 'etag': '"319579

{'kind': 'calendar#event', 'etag': '"3195790464392000"', 'id': 'cevnbep5ams3omfq3aiam5ui44', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=Y2V2bmJlcDVhbXMzb21mcTNhaWFtNXVpNDQgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:47:12.000Z', 'updated': '2020-08-20T03:47:12.196Z', 'summary': 'Sophomore Mentoring', 'description': 'Sophomore Mentoring', 'colorId': '7', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2020-09-19T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2020-09-19T14:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': 'cevnbep5ams3omfq3aiam5ui44@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2020-09-19 12:00:00 2020-09-19 14:00:00
{'kind': 'calendar#event', 'etag': '"31

{'kind': 'calendar#event', 'etag': '"3195790470742000"', 'id': '9mc9496k0ushqu2hp0q344i65s', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=OW1jOTQ5NmswdXNocXUyaHAwcTM0NGk2NXMgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:47:15.000Z', 'updated': '2020-08-20T03:47:15.371Z', 'summary': 'Senior Mentoring', 'description': 'Senior Mentoring', 'colorId': '9', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2020-09-26T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2020-09-26T14:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': '9mc9496k0ushqu2hp0q344i65s@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2020-09-26 10:00:00 2020-09-26 11:00:00
{'kind': 'calendar#event', 'etag': '"31957904

{'kind': 'calendar#event', 'etag': '"3195790476986000"', 'id': '35btmvf9ptgbto5h8pkrqhad1k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=MzVidG12ZjlwdGdidG81aDhwa3JxaGFkMWsgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:47:18.000Z', 'updated': '2020-08-20T03:47:18.493Z', 'summary': 'Math Matters', 'description': 'Math Matters', 'colorId': '3', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2020-10-17T11:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2020-10-17T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': '35btmvf9ptgbto5h8pkrqhad1k@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2020-10-17 10:00:00 2020-10-17 12:00:00
{'kind': 'calendar#event', 'etag': '"3195790477744000

...

{'kind': 'calendar#event', 'etag': '"3195790563910000"', 'id': 't1laco63v28cfrqbo7q6t36280', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=dDFsYWNvNjN2MjhjZnJxYm83cTZ0MzYyODAgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:48:01.000Z', 'updated': '2020-08-20T03:48:01.955Z', 'summary': 'EC Meeting', 'description': 'EC Meeting', 'colorId': '1', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2021-03-27T14:30:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2021-03-27T16:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': 't1laco63v28cfrqbo7q6t36280@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2021-03-27 11:00:00 2021-03-27 12:00:00
{'kind': 'calendar#event', 'etag': '"3195790564630000"', 

{'kind': 'calendar#event', 'etag': '"3195790570190000"', 'id': 'rbqlb1i72s54agoeii8301qjug', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=cmJxbGIxaTcyczU0YWdvZWlpODMwMXFqdWcgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:48:05.000Z', 'updated': '2020-08-20T03:48:05.095Z', 'summary': 'Test Prep', 'description': 'Test Prep', 'colorId': '5', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2021-04-03T10:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2021-04-03T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': 'rbqlb1i72s54agoeii8301qjug@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2021-04-03 10:00:00 2021-04-03 12:00:00
{'kind': 'calendar#event', 'etag': '"3195790570812000"', 'i

{'kind': 'calendar#event', 'etag': '"3195790576036000"', 'id': 'eknmiphmkvp1ihilhkhh5pch1c', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=ZWtubWlwaG1rdnAxaWhpbGhraGg1cGNoMWMgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:48:07.000Z', 'updated': '2020-08-20T03:48:08.018Z', 'summary': 'Sophomore Mentoring', 'description': 'Sophomore Mentoring', 'colorId': '7', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2021-04-17T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2021-04-17T14:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': 'eknmiphmkvp1ihilhkhh5pch1c@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2021-04-17 12:00:00 2021-04-17 14:00:00
{'kind': 'calendar#event', 'etag': '"31

{'kind': 'calendar#event', 'etag': '"3195790582122000"', 'id': '0pl7ugra9lqd2a36vnn3eog46s', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=MHBsN3VncmE5bHFkMmEzNnZubjNlb2c0NnMgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:48:11.000Z', 'updated': '2020-08-20T03:48:11.061Z', 'summary': 'Senior Mentoring', 'description': 'Senior Mentoring', 'colorId': '9', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2021-04-24T12:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2021-04-24T14:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': '0pl7ugra9lqd2a36vnn3eog46s@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2021-04-24 10:00:00 2021-04-24 11:00:00
{'kind': 'calendar#event', 'etag': '"31957905

{'kind': 'calendar#event', 'etag': '"3195790588210000"', 'id': 'b9vnqk2co6vsuf7rhflm83219g', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=Yjl2bnFrMmNvNnZzdWY3cmhmbG04MzIxOWcgY184MjJtMDdha3Rwam5lM2dmYnI3Y2o4Z2prc0Bn', 'created': '2020-08-20T03:48:14.000Z', 'updated': '2020-08-20T03:48:14.105Z', 'summary': 'EC Meeting', 'description': 'EC Meeting', 'colorId': '1', 'creator': {'email': 'admin@mindsmatterseattle.org'}, 'organizer': {'email': 'c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com', 'displayName': 'Minds Matter Seattle 2021-2022', 'self': True}, 'start': {'dateTime': '2021-05-08T14:30:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2021-05-08T16:00:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': 'b9vnqk2co6vsuf7rhflm83219g@google.com', 'sequence': 0, 'guestsCanInviteOthers': False, 'reminders': {'useDefault': True}}
2021-05-08 10:00:00 2021-05-08 11:00:00
{'kind': 'calendar#event', 'etag': '"3195790588926000"', 

In [72]:
df.to_csv('calendar_IDs.csv')

In [43]:
for k,row in df.iterrows():
    if 'ec' in row['Share with group']:
        event = eventservice.get(calendarId=calendarID, eventId=row.calendarEventID).execute()
        attendees=event.get('attendees',[])
        try:
            ecinvite=next(a for a in attendees if a['email']=='ec')
        except StopIteration:
            ecinvite = {
                'email': 'ec@mindsmatterseattle.org'
            }
            attendees.append(ecinvite)
        
        event['attendees'] =  attendees
        
        eventservice.update(calendarId=calendarID, eventId=row.calendarEventID, body=event).execute()
   

In [80]:
for k,row in df.iterrows():
    emails=map(lambda x: x.replace(' ','') ,row['Share with group'].split(','))
    event = eventservice.get(calendarId=calendarID, eventId=row.calendarEventID).execute()
    attendees=event.get('attendees',[])
    for email in emails:
        try:
            invite=next(a for a in attendees if a['email']==email + "@mindsmatterseattle.org")
        except StopIteration:
            print(f'inviting {email} to {event["summary"]}')
            invite = {
                'email': f'{email}@mindsmatterseattle.org'
            }
            attendees.append(invite)

    event['attendees'] =  attendees 
    eventservice.update(calendarId=calendarID, eventId=row.calendarEventID, body=event).execute()
   

inviting ec to Operations Retreat
inviting ec to Pre-kickoff EC meeting
inviting students2023 to Student Welcome
inviting students2022 to Student Welcome
inviting students2021 to Student Welcome
inviting ec to Student Welcome
inviting volunteers to Volunteer Orientation
inviting ec to Volunteer Orientation
inviting students2023 to Sophomore Orientation
inviting students2022 to Junior Orientation
inviting students2021 to Senior Orientation
inviting math-instructors to Math Matters
inviting students2023 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2022 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2021 to Senior Enrichment
inviting 2023mentors to Sophomore Mentoring
inviting students2023 to Sophomore Mentoring
inviting 2022mentors to Junior Mentoring
inviting students2022 to Junior Mentoring
inviting 2021mentors to Senior Mentoring
inviting students2021 to Senior Mentoring
inviting wct-instructors to Writing and C

inviting testprep-instructors to Test Prep
inviting students2022 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2021 to Senior Enrichment
inviting 2023mentors to Sophomore Mentoring
inviting students2023 to Sophomore Mentoring
inviting 2022mentors to Junior Mentoring
inviting students2022 to Junior Mentoring
inviting 2021mentors to Senior Mentoring
inviting students2021 to Senior Mentoring
inviting wct-instructors to Writing and Critical Thinking
inviting students2023 to Writing and Critical Thinking
inviting math-instructors to Math Matters
inviting students2023 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2022 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2021 to Senior Enrichment
inviting 2023mentors to Sophomore Mentoring
inviting students2023 to Sophomore Mentoring
inviting 2022mentors to Junior Mentoring
inviting students2022 to Junior Mentoring
inviting 2021mento

inviting ec to EC Meeting
inviting wct-instructors to Writing and Critical Thinking
inviting students2023 to Writing and Critical Thinking
inviting math-instructors to Math Matters
inviting students2023 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2022 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2021 to Senior Enrichment
inviting 2023mentors to Sophomore Mentoring
inviting students2023 to Sophomore Mentoring
inviting 2022mentors to Junior Mentoring
inviting students2022 to Junior Mentoring
inviting 2021mentors to Senior Mentoring
inviting students2021 to Senior Mentoring
inviting active to Graduation


In [9]:
df.to_csv('calendar_events.csv')

In [25]:
np.isnan(df.loc[244, 'calendarEventID'])

True